In [2]:
! conda --version

conda 4.7.5


## Beysian Inference
It is a method of inference where the probibility of a hypothesis is updated as new evidence becomes available
### Steps

** 1. Begin with a prior distribution processes : P(H)

** 2. collect data to obtain the observed distribution P(E) as evidence

** 3. calculate the likelihood of our observation given the hypothesis distribution (you can say we want to know how likely we can generate these observations data given aour hypothesis
distribution P(E|H))   

** 4. obtain the posterior or simple update the prior which is the correct distribution for another unseen data) P(H|E)

### bayes rule
$$
P(H|E) = \frac{P(E|H) \times P(H)}{p(E)} 
$$

## Maximum a posterior estimation

**note : the likelihood of a given sample $x$ is calculated as $p(x|\mu,\sigma) = \frac{1}{\sqrt{(2\pi \sigma^{2})}} \exp(-\frac{(x-\mu)^{2}}{2\sigma^{2}}) $

**example: we have two class of male and female and their height---> we can compute each class mean and std and then we can create our prior 

**now if given a new sample we can predict its class therefore we can do it for unseen samples

**The problem is as 

$$
Class = \armax_{c \in {c_{male},c_{female} } } P(c|x)
$$

therefor using bayes Rule and theank to likelihood that we can compute $p(x|c)$ we can reformulate and write general expression

$$
\argmax_{c\in C} \frac{p(x|c)\times p(c)}{p(x)}
$$

However we don't have $p(x)$ because we haven't seen all the x yet and there are a lots of unseen $x$ yet, but if you see the optimization it is independent from x, or better to say 
the probability distribution of data is independent from our classess because the data does not know and care about being male or female so we simply can forget about that in our optimization pronblem

*So

$
\argmax_{c\in C} p(x|c)\times p(c)
$

## lets try the Bayes Inference from Scratch

In [5]:
!git clone https://github.com/saeedahmadian/sample_data.git


Cloning into 'sample_data'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [6]:
df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .option('nanValue', ' ')\
  .option('nullValue', ' ')\
  .load('sample_data/weight-height.csv')

In [7]:
df_data.show()

+------+----------------+----------------+
|Gender|          Height|          Weight|
+------+----------------+----------------+
|  Male| 73.847017017515|241.893563180437|
|  Male|68.7819040458903|  162.3104725213|
|  Male|74.1101053917849|  212.7408555565|
|  Male|71.7309784033377|220.042470303077|
|  Male|69.8817958611153|206.349800623871|
|  Male|67.2530156878065|152.212155757083|
|  Male|68.7850812516616|183.927888604031|
|  Male|68.3485155115879|167.971110489509|
|  Male| 67.018949662883| 175.92944039571|
|  Male|63.4564939783664|156.399676387112|
|  Male|71.1953822829745|186.604925560358|
|  Male|71.6408051192206|213.741169489411|
|  Male|64.7663291334055|167.127461073476|
|  Male|69.2830700967204|189.446181386738|
|  Male|69.2437322298112|186.434168021239|
|  Male|67.6456197004212|172.186930058117|
|  Male|72.4183166259878|196.028506330482|
|  Male| 63.974325721061| 172.88347020878|
|  Male|69.6400598997523| 185.98395757313|
|  Male|67.9360048540095|182.426648013226|
+------+---

In [8]:
from pyspark.ml.feature import StringIndexer
stind = StringIndexer(inputCol='Gender',outputCol='Sex')
data= stind.fit(df_data).transform(df_data)
data.show()

+------+----------------+----------------+---+
|Gender|          Height|          Weight|Sex|
+------+----------------+----------------+---+
|  Male| 73.847017017515|241.893563180437|0.0|
|  Male|68.7819040458903|  162.3104725213|0.0|
|  Male|74.1101053917849|  212.7408555565|0.0|
|  Male|71.7309784033377|220.042470303077|0.0|
|  Male|69.8817958611153|206.349800623871|0.0|
|  Male|67.2530156878065|152.212155757083|0.0|
|  Male|68.7850812516616|183.927888604031|0.0|
|  Male|68.3485155115879|167.971110489509|0.0|
|  Male| 67.018949662883| 175.92944039571|0.0|
|  Male|63.4564939783664|156.399676387112|0.0|
|  Male|71.1953822829745|186.604925560358|0.0|
|  Male|71.6408051192206|213.741169489411|0.0|
|  Male|64.7663291334055|167.127461073476|0.0|
|  Male|69.2830700967204|189.446181386738|0.0|
|  Male|69.2437322298112|186.434168021239|0.0|
|  Male|67.6456197004212|172.186930058117|0.0|
|  Male|72.4183166259878|196.028506330482|0.0|
|  Male| 63.974325721061| 172.88347020878|0.0|
|  Male|69.64

In [13]:
df=data.sample(.1,False)

In [14]:
import numpy as np
npdata = np.array(df.select('Height','Weight','Sex').collect())
npdata.shape

(1022, 3)

In [25]:
from sklearn.model_selection import train_test_split
features = npdata[:,[0,1]]
labels = npdata[:,2]
feature_train, feature_test, label_train, label_test = train_test_split(features,labels,test_size=0.2,random_state=100)

In [19]:
labels.size

1022

In [21]:
def getpriors(labels):
    priors ={}
    for classname in labels:
        N = labels.size
        classoccurence = (labels==classname).sum()
        priors[classname] = classoccurence/N
    return priors

priors= getpriors(labels)
priors        
        

{0.0: 0.48043052837573386, 1.0: 0.5195694716242661}

In [22]:
uniqueclass = np.unique(labels)
uniqueclass

array([0., 1.])

In [26]:
def fit(features,labels):
    uniqueclass = np.unique(labels)
    coeffs ={}
    for classname in uniqueclass:
        featuresinclass= features[labels==classname]
        statsinclass = [(np.mean(feature), np.std(feature)) for feature in zip(*featuresinclass)]
        coeffs[classname] =statsinclass
    return coeffs
    
model =fit(feature_train,label_train)
model

{0.0: [(69.13403554077037, 2.930658926690435),
  (187.37257971197354, 20.83408798298229)],
 1.0: [(63.705606774791775, 2.6222036997974185),
  (136.3859698876928, 19.118292943402885)]}

In [32]:
import math
def getlikelihood(x,featureind,model,classname):
    classstats= model[classname]
    mean = classstats[featureind][0]
    std = classstats[featureind][1]
    f = (1/(np.sqrt(2*math.pi*pow(std,2))))* np.exp(-pow((x-mean),2)/(2*pow(std,2)))
    return f
x =np.array([69]) #weight index is 0 so-->
likelihood_x_belongs_to_male =getlikelihood(x,0,model,0)
likelihood_x_belongs_to_male

array([0.13598486])